### CIFAR 10 Classifier
- Ref: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html


In [48]:
import os
import time
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.optim as optim
import numpy as np

In [69]:
# 变量
EPOCHS = 10
BATCH_SIZE = 128
learning_rate = 0.05
momentum = 0.9
DATA_DIR = '../data'
classes = ('plane', 'car', 'bird', 'cat',  # 10 个分类
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
criterion = nn.CrossEntropyLoss()
device = torch.device("mps")

In [50]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [71]:
class ConvNetV2(nn.Module):
    """
    Using a sequential CNN that has more layers and a larger batch size along with defined flattening layer
    """
    def __init__(self):
        super(ConvNetV2, self).__init__()
        # convolutional layer
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # fully connected layers
        self.fc1 = nn.Linear(64 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 64)
        self.fc3 = nn.Linear(64, 10)
        # dropout
        self.dropout = nn.Dropout(p=.5)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flattening
        x = x.view(-1, 64 * 4 * 4)
        # fully connected layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [75]:
class ConvNetV3(nn.Module):
    """
    - increased the amount of layers, added Convolutional blocks that have a kernel size of 3. 
    - A kernel is a filter used to extract features from the images. 
    - changed the channel sizes.
    """
    def __init__(self):
        super(ConvNetV3, self).__init__()
        self.conv_layer = nn.Sequential(
            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),
            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        """
        Perform forward.
        """
        # conv layers
        x = self.conv_layer(x)
        # flatten
        x = x.view(x.size(0), -1)
        # fc layer
        x = self.fc_layer(x)
        return x

In [51]:
# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [65]:
def train(model, device, train_loader, optimizer, epoch):
    """
    训练模型
    """
    model.train()
    epoch_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        # 将变量转移至 MPS
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # loss = F.nll_loss(output, target)  # 损失函数
        loss = criterion(output, target)  # 损失函数

        loss.backward()  # 求导
        optimizer.step()
        if batch_idx % 10 > 0:
            continue
        # 训练准确率
        pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        # _, pred = torch.max(output.data, 1)
        # print(f"pred: {pred}")
        correct = pred.eq(target.view_as(pred)).sum().item()
        # print(f"correct: {correct}")
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.6f}%\tCosts: {:.6f} seconds'.format(
            epoch, 
            batch_idx * len(data), 
            len(train_loader.dataset),
            batch_idx / len(train_loader) * 100, 
            loss.item(), 
            correct / len(data) * 100,
            time.time() - epoch_time
        ))
        epoch_time = time.time()

In [66]:
def test(model, device, test_loader):
    """
    测试模型
    """
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():  # 不需要求导
        for data, target in test_loader:
            # 批量处理
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [72]:
def main(Model=ConvNet):
    # 归一化
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    will_download = not os.path.exists(f"{DATA_DIR}/cifar-10-python.tar.gz")
    train_set = torchvision.datasets.CIFAR10(
        root=DATA_DIR, train=True, download=will_download, transform=transform
    )
    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2
    )
    test_set = torchvision.datasets.CIFAR10(
        root=DATA_DIR, train=False, download=will_download, transform=transform
    )
    test_loader = torch.utils.data.DataLoader(
        test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2
    )

    model = Model().to(device)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

    for epoch in range(EPOCHS):
        train(model, device, train_loader, optimizer, epoch + 1)
        test(model, device, test_loader)

In [70]:
# v1: Test set: Average loss: 0.0094, Accuracy: 6113/10000 (61%)
main()

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.304431	Accuracy: 10.156250%	Costs: 3.135964 seconds
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.300719	Accuracy: 14.062500%	Costs: 0.071200 seconds
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.300053	Accuracy: 5.468750%	Costs: 0.060846 seconds
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.281366	Accuracy: 15.625000%	Costs: 0.060142 seconds
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.224038	Accuracy: 17.187500%	Costs: 0.063158 seconds
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.138773	Accuracy: 21.875000%	Costs: 0.060523 seconds
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.178654	Accuracy: 21.093750%	Costs: 0.059500 seconds
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.027178	Accuracy: 23.437500%	Costs: 0.059909 seconds
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.942906	Accuracy: 21.875000%	Costs: 0.062071 seconds
Train Epoch: 1 [11520/50000 (23%)]	Loss: 1.987103	Accuracy: 27.343750%	Costs: 0.066109 seconds
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.976597	Accuracy: 25.000

Train Epoch: 3 [8960/50000 (18%)]	Loss: 1.180305	Accuracy: 54.687500%	Costs: 0.065854 seconds
Train Epoch: 3 [10240/50000 (20%)]	Loss: 1.370075	Accuracy: 49.218750%	Costs: 0.064539 seconds
Train Epoch: 3 [11520/50000 (23%)]	Loss: 1.105427	Accuracy: 61.718750%	Costs: 0.062928 seconds
Train Epoch: 3 [12800/50000 (26%)]	Loss: 1.300483	Accuracy: 60.937500%	Costs: 0.062818 seconds
Train Epoch: 3 [14080/50000 (28%)]	Loss: 1.436447	Accuracy: 46.875000%	Costs: 0.068301 seconds
Train Epoch: 3 [15360/50000 (31%)]	Loss: 1.281343	Accuracy: 54.687500%	Costs: 0.085975 seconds
Train Epoch: 3 [16640/50000 (33%)]	Loss: 1.396370	Accuracy: 51.562500%	Costs: 0.128013 seconds
Train Epoch: 3 [17920/50000 (36%)]	Loss: 1.182170	Accuracy: 62.500000%	Costs: 0.076349 seconds
Train Epoch: 3 [19200/50000 (38%)]	Loss: 1.248197	Accuracy: 60.937500%	Costs: 0.063112 seconds
Train Epoch: 3 [20480/50000 (41%)]	Loss: 1.425209	Accuracy: 47.656250%	Costs: 0.063775 seconds
Train Epoch: 3 [21760/50000 (43%)]	Loss: 1.198910	A

Train Epoch: 5 [20480/50000 (41%)]	Loss: 1.169727	Accuracy: 57.812500%	Costs: 0.066905 seconds
Train Epoch: 5 [21760/50000 (43%)]	Loss: 1.019143	Accuracy: 66.406250%	Costs: 0.080685 seconds
Train Epoch: 5 [23040/50000 (46%)]	Loss: 0.955044	Accuracy: 63.281250%	Costs: 0.062700 seconds
Train Epoch: 5 [24320/50000 (49%)]	Loss: 1.351006	Accuracy: 47.656250%	Costs: 0.058396 seconds
Train Epoch: 5 [25600/50000 (51%)]	Loss: 0.972339	Accuracy: 63.281250%	Costs: 0.059868 seconds
Train Epoch: 5 [26880/50000 (54%)]	Loss: 1.119849	Accuracy: 60.937500%	Costs: 0.061005 seconds
Train Epoch: 5 [28160/50000 (56%)]	Loss: 1.010540	Accuracy: 60.937500%	Costs: 0.058459 seconds
Train Epoch: 5 [29440/50000 (59%)]	Loss: 0.959473	Accuracy: 63.281250%	Costs: 0.058529 seconds
Train Epoch: 5 [30720/50000 (61%)]	Loss: 1.098511	Accuracy: 57.812500%	Costs: 0.060392 seconds
Train Epoch: 5 [32000/50000 (64%)]	Loss: 0.959183	Accuracy: 64.843750%	Costs: 0.058388 seconds
Train Epoch: 5 [33280/50000 (66%)]	Loss: 1.122193	

Train Epoch: 7 [30720/50000 (61%)]	Loss: 0.999165	Accuracy: 64.843750%	Costs: 0.059624 seconds
Train Epoch: 7 [32000/50000 (64%)]	Loss: 1.045777	Accuracy: 62.500000%	Costs: 0.058855 seconds
Train Epoch: 7 [33280/50000 (66%)]	Loss: 0.913430	Accuracy: 69.531250%	Costs: 0.059359 seconds
Train Epoch: 7 [34560/50000 (69%)]	Loss: 0.764908	Accuracy: 75.000000%	Costs: 0.059288 seconds
Train Epoch: 7 [35840/50000 (72%)]	Loss: 1.246282	Accuracy: 57.812500%	Costs: 0.059484 seconds
Train Epoch: 7 [37120/50000 (74%)]	Loss: 1.061205	Accuracy: 60.937500%	Costs: 0.062762 seconds
Train Epoch: 7 [38400/50000 (77%)]	Loss: 1.026763	Accuracy: 56.250000%	Costs: 0.059785 seconds
Train Epoch: 7 [39680/50000 (79%)]	Loss: 1.217953	Accuracy: 61.718750%	Costs: 0.057711 seconds
Train Epoch: 7 [40960/50000 (82%)]	Loss: 0.851217	Accuracy: 74.218750%	Costs: 0.059321 seconds
Train Epoch: 7 [42240/50000 (84%)]	Loss: 1.330067	Accuracy: 57.031250%	Costs: 0.058341 seconds
Train Epoch: 7 [43520/50000 (87%)]	Loss: 1.084240	

Train Epoch: 9 [40960/50000 (82%)]	Loss: 1.166059	Accuracy: 56.250000%	Costs: 0.058545 seconds
Train Epoch: 9 [42240/50000 (84%)]	Loss: 0.873923	Accuracy: 69.531250%	Costs: 0.058166 seconds
Train Epoch: 9 [43520/50000 (87%)]	Loss: 1.028494	Accuracy: 67.968750%	Costs: 0.057828 seconds
Train Epoch: 9 [44800/50000 (90%)]	Loss: 1.093607	Accuracy: 61.718750%	Costs: 0.057770 seconds
Train Epoch: 9 [46080/50000 (92%)]	Loss: 1.038171	Accuracy: 57.031250%	Costs: 0.057831 seconds
Train Epoch: 9 [47360/50000 (95%)]	Loss: 0.960914	Accuracy: 66.406250%	Costs: 0.058235 seconds
Train Epoch: 9 [48640/50000 (97%)]	Loss: 0.819378	Accuracy: 70.312500%	Costs: 0.057678 seconds
Train Epoch: 9 [31200/50000 (100%)]	Loss: 1.058603	Accuracy: 61.250000%	Costs: 0.056213 seconds

Test set: Average loss: 0.0096, Accuracy: 5955/10000 (60%)

Train Epoch: 10 [0/50000 (0%)]	Loss: 0.751707	Accuracy: 73.437500%	Costs: 3.141622 seconds
Train Epoch: 10 [1280/50000 (3%)]	Loss: 1.049131	Accuracy: 61.718750%	Costs: 0.077391 s

In [73]:
# v2: Test set: Average loss: 0.0072, Accuracy: 6893/10000 (69%)
learning_rate = 0.01
main(ConvNetV2)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.301452	Accuracy: 15.625000%	Costs: 5.554205 seconds
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.298547	Accuracy: 10.937500%	Costs: 0.199547 seconds
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.319240	Accuracy: 5.468750%	Costs: 0.109542 seconds
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.295460	Accuracy: 14.062500%	Costs: 0.102183 seconds
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.287434	Accuracy: 13.281250%	Costs: 0.100198 seconds
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.252058	Accuracy: 13.281250%	Costs: 0.104099 seconds
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.289289	Accuracy: 12.500000%	Costs: 0.103629 seconds
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.176174	Accuracy: 14.843750%	Costs: 0.100823 seconds
Train Epoch: 1 [10240/50000 (20%)]	Loss: 2.169571	Accuracy: 17.968750%	Costs: 0.104426 seconds
Train Epoch: 1 [11520/50000 (23%)]	Loss: 2.108824	Accuracy: 17.187500%	Costs: 0.100559 seconds
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.135642	Accuracy: 14.843

Train Epoch: 3 [7680/50000 (15%)]	Loss: 1.369048	Accuracy: 47.656250%	Costs: 0.099275 seconds
Train Epoch: 3 [8960/50000 (18%)]	Loss: 1.312048	Accuracy: 59.375000%	Costs: 0.099473 seconds
Train Epoch: 3 [10240/50000 (20%)]	Loss: 1.282092	Accuracy: 57.812500%	Costs: 0.099022 seconds
Train Epoch: 3 [11520/50000 (23%)]	Loss: 1.268891	Accuracy: 48.437500%	Costs: 0.099130 seconds
Train Epoch: 3 [12800/50000 (26%)]	Loss: 1.215062	Accuracy: 51.562500%	Costs: 0.099320 seconds
Train Epoch: 3 [14080/50000 (28%)]	Loss: 1.247858	Accuracy: 52.343750%	Costs: 0.098927 seconds
Train Epoch: 3 [15360/50000 (31%)]	Loss: 1.275908	Accuracy: 57.031250%	Costs: 0.099045 seconds
Train Epoch: 3 [16640/50000 (33%)]	Loss: 1.220633	Accuracy: 55.468750%	Costs: 0.099481 seconds
Train Epoch: 3 [17920/50000 (36%)]	Loss: 1.306697	Accuracy: 53.906250%	Costs: 0.098732 seconds
Train Epoch: 3 [19200/50000 (38%)]	Loss: 1.247440	Accuracy: 57.812500%	Costs: 0.098820 seconds
Train Epoch: 3 [20480/50000 (41%)]	Loss: 1.308869	Ac

Train Epoch: 5 [15360/50000 (31%)]	Loss: 0.984581	Accuracy: 62.500000%	Costs: 0.098951 seconds
Train Epoch: 5 [16640/50000 (33%)]	Loss: 1.298714	Accuracy: 57.031250%	Costs: 0.099366 seconds
Train Epoch: 5 [17920/50000 (36%)]	Loss: 1.168133	Accuracy: 57.812500%	Costs: 0.098768 seconds
Train Epoch: 5 [19200/50000 (38%)]	Loss: 1.061849	Accuracy: 66.406250%	Costs: 0.098589 seconds
Train Epoch: 5 [20480/50000 (41%)]	Loss: 0.966980	Accuracy: 71.093750%	Costs: 0.098629 seconds
Train Epoch: 5 [21760/50000 (43%)]	Loss: 1.032567	Accuracy: 67.968750%	Costs: 0.098622 seconds
Train Epoch: 5 [23040/50000 (46%)]	Loss: 1.063609	Accuracy: 60.937500%	Costs: 0.098431 seconds
Train Epoch: 5 [24320/50000 (49%)]	Loss: 1.244108	Accuracy: 65.625000%	Costs: 0.098752 seconds
Train Epoch: 5 [25600/50000 (51%)]	Loss: 1.196734	Accuracy: 63.281250%	Costs: 0.098522 seconds
Train Epoch: 5 [26880/50000 (54%)]	Loss: 1.253649	Accuracy: 57.812500%	Costs: 0.098385 seconds
Train Epoch: 5 [28160/50000 (56%)]	Loss: 0.934181	

Train Epoch: 7 [23040/50000 (46%)]	Loss: 1.000443	Accuracy: 71.093750%	Costs: 0.101030 seconds
Train Epoch: 7 [24320/50000 (49%)]	Loss: 1.158777	Accuracy: 64.062500%	Costs: 0.100006 seconds
Train Epoch: 7 [25600/50000 (51%)]	Loss: 0.986824	Accuracy: 66.406250%	Costs: 0.100344 seconds
Train Epoch: 7 [26880/50000 (54%)]	Loss: 0.934676	Accuracy: 64.062500%	Costs: 0.101202 seconds
Train Epoch: 7 [28160/50000 (56%)]	Loss: 0.964622	Accuracy: 72.656250%	Costs: 0.104933 seconds
Train Epoch: 7 [29440/50000 (59%)]	Loss: 1.082176	Accuracy: 63.281250%	Costs: 0.103870 seconds
Train Epoch: 7 [30720/50000 (61%)]	Loss: 1.017952	Accuracy: 67.187500%	Costs: 0.102483 seconds
Train Epoch: 7 [32000/50000 (64%)]	Loss: 1.114715	Accuracy: 60.937500%	Costs: 0.099642 seconds
Train Epoch: 7 [33280/50000 (66%)]	Loss: 1.123822	Accuracy: 65.625000%	Costs: 0.100483 seconds
Train Epoch: 7 [34560/50000 (69%)]	Loss: 0.828795	Accuracy: 66.406250%	Costs: 0.099791 seconds
Train Epoch: 7 [35840/50000 (72%)]	Loss: 0.992060	

Train Epoch: 9 [33280/50000 (66%)]	Loss: 0.974424	Accuracy: 67.968750%	Costs: 0.102724 seconds
Train Epoch: 9 [34560/50000 (69%)]	Loss: 0.731521	Accuracy: 75.781250%	Costs: 0.101120 seconds
Train Epoch: 9 [35840/50000 (72%)]	Loss: 0.946236	Accuracy: 68.750000%	Costs: 0.099893 seconds
Train Epoch: 9 [37120/50000 (74%)]	Loss: 1.116344	Accuracy: 60.156250%	Costs: 0.099766 seconds
Train Epoch: 9 [38400/50000 (77%)]	Loss: 0.929531	Accuracy: 74.218750%	Costs: 0.099475 seconds
Train Epoch: 9 [39680/50000 (79%)]	Loss: 0.973138	Accuracy: 68.750000%	Costs: 0.101191 seconds
Train Epoch: 9 [40960/50000 (82%)]	Loss: 1.012279	Accuracy: 67.187500%	Costs: 0.103397 seconds
Train Epoch: 9 [42240/50000 (84%)]	Loss: 0.882653	Accuracy: 72.656250%	Costs: 0.103695 seconds
Train Epoch: 9 [43520/50000 (87%)]	Loss: 1.147050	Accuracy: 64.843750%	Costs: 0.103711 seconds
Train Epoch: 9 [44800/50000 (90%)]	Loss: 1.067288	Accuracy: 64.062500%	Costs: 0.100583 seconds
Train Epoch: 9 [46080/50000 (92%)]	Loss: 0.918998	

In [76]:
# v3: Test set: Average loss: 0.0069, Accuracy: 7126/10000 (71%)
learning_rate = 0.001
main(ConvNetV3)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.301452	Accuracy: 14.843750%	Costs: 5.690009 seconds
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.305266	Accuracy: 8.593750%	Costs: 0.991495 seconds
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.299646	Accuracy: 10.156250%	Costs: 0.972018 seconds
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.289247	Accuracy: 15.625000%	Costs: 0.969219 seconds
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.286820	Accuracy: 12.500000%	Costs: 0.976906 seconds
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.283652	Accuracy: 11.718750%	Costs: 0.965818 seconds
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.273756	Accuracy: 20.312500%	Costs: 0.987895 seconds
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.269750	Accuracy: 16.406250%	Costs: 0.974843 seconds
Train Epoch: 1 [10240/50000 (20%)]	Loss: 2.274270	Accuracy: 12.500000%	Costs: 0.989040 seconds
Train Epoch: 1 [11520/50000 (23%)]	Loss: 2.263450	Accuracy: 15.625000%	Costs: 1.002996 seconds
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.240760	Accuracy: 28.906

Train Epoch: 3 [7680/50000 (15%)]	Loss: 1.332086	Accuracy: 53.906250%	Costs: 0.967604 seconds
Train Epoch: 3 [8960/50000 (18%)]	Loss: 1.239973	Accuracy: 60.156250%	Costs: 0.962218 seconds
Train Epoch: 3 [10240/50000 (20%)]	Loss: 1.195588	Accuracy: 53.125000%	Costs: 0.963983 seconds
Train Epoch: 3 [11520/50000 (23%)]	Loss: 1.191583	Accuracy: 57.812500%	Costs: 0.962662 seconds
Train Epoch: 3 [12800/50000 (26%)]	Loss: 1.294580	Accuracy: 52.343750%	Costs: 0.962838 seconds
Train Epoch: 3 [14080/50000 (28%)]	Loss: 1.249730	Accuracy: 51.562500%	Costs: 0.961626 seconds
Train Epoch: 3 [15360/50000 (31%)]	Loss: 1.271724	Accuracy: 53.125000%	Costs: 0.961653 seconds
Train Epoch: 3 [16640/50000 (33%)]	Loss: 1.265711	Accuracy: 58.593750%	Costs: 0.966371 seconds
Train Epoch: 3 [17920/50000 (36%)]	Loss: 1.306460	Accuracy: 52.343750%	Costs: 0.961135 seconds
Train Epoch: 3 [19200/50000 (38%)]	Loss: 1.139387	Accuracy: 60.937500%	Costs: 0.978135 seconds
Train Epoch: 3 [20480/50000 (41%)]	Loss: 1.142840	Ac

Train Epoch: 5 [15360/50000 (31%)]	Loss: 0.904313	Accuracy: 71.875000%	Costs: 0.964381 seconds
Train Epoch: 5 [16640/50000 (33%)]	Loss: 0.840529	Accuracy: 67.187500%	Costs: 0.991121 seconds
Train Epoch: 5 [17920/50000 (36%)]	Loss: 0.988583	Accuracy: 68.750000%	Costs: 0.978948 seconds
Train Epoch: 5 [19200/50000 (38%)]	Loss: 0.889934	Accuracy: 64.062500%	Costs: 0.986213 seconds
Train Epoch: 5 [20480/50000 (41%)]	Loss: 0.739540	Accuracy: 76.562500%	Costs: 1.002816 seconds
Train Epoch: 5 [21760/50000 (43%)]	Loss: 0.881534	Accuracy: 62.500000%	Costs: 0.988742 seconds
Train Epoch: 5 [23040/50000 (46%)]	Loss: 0.910425	Accuracy: 66.406250%	Costs: 0.969704 seconds
Train Epoch: 5 [24320/50000 (49%)]	Loss: 0.895967	Accuracy: 63.281250%	Costs: 0.985183 seconds
Train Epoch: 5 [25600/50000 (51%)]	Loss: 0.757031	Accuracy: 70.312500%	Costs: 0.977671 seconds
Train Epoch: 5 [26880/50000 (54%)]	Loss: 1.035573	Accuracy: 64.062500%	Costs: 0.981698 seconds
Train Epoch: 5 [28160/50000 (56%)]	Loss: 0.672836	

Train Epoch: 7 [23040/50000 (46%)]	Loss: 0.774647	Accuracy: 67.968750%	Costs: 0.963968 seconds
Train Epoch: 7 [24320/50000 (49%)]	Loss: 0.817063	Accuracy: 67.968750%	Costs: 0.973721 seconds
Train Epoch: 7 [25600/50000 (51%)]	Loss: 0.698120	Accuracy: 77.343750%	Costs: 0.964110 seconds
Train Epoch: 7 [26880/50000 (54%)]	Loss: 0.762784	Accuracy: 69.531250%	Costs: 0.964557 seconds
Train Epoch: 7 [28160/50000 (56%)]	Loss: 0.690007	Accuracy: 70.312500%	Costs: 0.968636 seconds
Train Epoch: 7 [29440/50000 (59%)]	Loss: 0.834234	Accuracy: 72.656250%	Costs: 0.968402 seconds
Train Epoch: 7 [30720/50000 (61%)]	Loss: 0.733666	Accuracy: 74.218750%	Costs: 0.964154 seconds
Train Epoch: 7 [32000/50000 (64%)]	Loss: 0.585054	Accuracy: 78.125000%	Costs: 1.045373 seconds
Train Epoch: 7 [33280/50000 (66%)]	Loss: 0.797808	Accuracy: 69.531250%	Costs: 0.966322 seconds
Train Epoch: 7 [34560/50000 (69%)]	Loss: 0.667969	Accuracy: 71.875000%	Costs: 0.968613 seconds
Train Epoch: 7 [35840/50000 (72%)]	Loss: 0.712619	

Train Epoch: 9 [30720/50000 (61%)]	Loss: 0.597213	Accuracy: 77.343750%	Costs: 0.964955 seconds
Train Epoch: 9 [32000/50000 (64%)]	Loss: 0.556265	Accuracy: 80.468750%	Costs: 0.965595 seconds
Train Epoch: 9 [33280/50000 (66%)]	Loss: 0.535153	Accuracy: 82.031250%	Costs: 0.965715 seconds
Train Epoch: 9 [34560/50000 (69%)]	Loss: 0.529297	Accuracy: 82.031250%	Costs: 0.965978 seconds
Train Epoch: 9 [35840/50000 (72%)]	Loss: 0.540064	Accuracy: 82.031250%	Costs: 0.965963 seconds
Train Epoch: 9 [37120/50000 (74%)]	Loss: 0.479728	Accuracy: 86.718750%	Costs: 0.965844 seconds
Train Epoch: 9 [38400/50000 (77%)]	Loss: 0.496421	Accuracy: 80.468750%	Costs: 0.965765 seconds
Train Epoch: 9 [39680/50000 (79%)]	Loss: 0.562054	Accuracy: 81.250000%	Costs: 0.965549 seconds
Train Epoch: 9 [40960/50000 (82%)]	Loss: 0.502290	Accuracy: 78.906250%	Costs: 0.964345 seconds
Train Epoch: 9 [42240/50000 (84%)]	Loss: 0.511117	Accuracy: 78.125000%	Costs: 0.964745 seconds
Train Epoch: 9 [43520/50000 (87%)]	Loss: 0.610823	